In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

import cv2
from sklearn.model_selection import KFold

from keras.models import Model, Sequential
from keras.applications import MobileNet
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Reshape, Conv2D, Convolution2D, Activation
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Using TensorFlow backend.


In [2]:
train = pd.read_json("_RawData/train.json/data/processed/train.json")
test = pd.read_json("_RawData/test.json/data/processed/test.json")


In [3]:
# Train data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train['band_2']])

X_train = np.concatenate([x_band1[:, :, :, np.newaxis],
                          x_band2[:, :, :, np.newaxis],
                          ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)

target_train=train['is_iceberg']

del train

In [4]:
# Test data
x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_1']])
x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test['band_2']])

X_test = np.concatenate([x_band1[:, :, :, np.newaxis],
                         x_band2[:, :, :, np.newaxis],
                         ((x_band1+x_band1)/2)[:, :, :, np.newaxis]], axis=-1)

id_test = test['id'].values

del test; del x_band1; del x_band2

In [8]:
# Define CNN Model Architecture (Kaggle can't access the weights file)
img_height = 75
img_width = 75
img_channels = 3
img_dim = (img_height, img_width, img_channels)

def inceptionv3(img_dim=img_dim):
    model = Sequential()

    model.add(Convolution2D(96, 3, 3, border_mode = 'same', input_shape=img_dim))
    model.add(Activation('relu'))
    model.add(Convolution2D(96, 3, 3,border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(96, 3, 3, border_mode='same', subsample = (2,2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(192, 3, 3, border_mode = 'same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(192, 3, 3,border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(192, 3, 3,border_mode='same', subsample = (2,2)))
    model.add(Dropout(0.5))

    model.add(Convolution2D(192, 3, 3, border_mode = 'same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(192, 1, 1,border_mode='valid'))
    model.add(Activation('relu'))
    model.add(Convolution2D(2, 1, 1, border_mode='valid'))



    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='sigmoid'))
#     model.add(Activation('sigmoid'))
#     model = make_parallel(model, 4)

#     x = Reshape((1, 1, 1024), name='reshape_1')(x)
#     x = Dropout(0.2, name='dropout')(x)
#     x = Conv2D(2, (1, 1),
#                padding='same', name='conv_preds')(x)
#     x = Dense('sigmoid', name='act_softmax')(x)
#     x = Reshape((classes,), name='reshape_2')(x)
    
#     bn = BatchNormalization()(input_tensor)
#     x = base_model(bn)
#     x = GlobalAveragePooling2D()(x)
#     x = Dropout(0.5)(x)
#     output = Dense(1, activation='sigmoid')(x)
#     model = Model(input_tensor, output)
    
    return model

model = inceptionv3()
model.summary()

C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3), padding="same", input_shape=(75, 75, 3...)`
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3), padding="same")`
  if sys.path[0] == '':
C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (3, 3), padding="same", strides=(2, 2))`
  
C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(192, (3, 3), padding="same")`
C:\Users\Dave\Anaconda3\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:19: UserWarning: Update your `Conv2D` call to the Keras

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 75, 75, 96)        2688      
_________________________________________________________________
activation_7 (Activation)    (None, 75, 75, 96)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 75, 75, 96)        83040     
_________________________________________________________________
activation_8 (Activation)    (None, 75, 75, 96)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 38, 38, 96)        83040     
_________________________________________________________________
dropout_3 (Dropout)          (None, 38, 38, 96)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 38, 38, 192)       166080    
__________

In [9]:
# Train Model and predict
def train_model(model, batch_size, epochs, img_size, x, y, test, n_fold, kf):
        
    train_scores = []; valid_scores = []
    preds_test = np.zeros(len(test), dtype = np.float)

    i = 1

    for train_index, test_index in kf.split(x):
        x_train = x[train_index]; x_valid = x[test_index]
        y_train = y[train_index]; y_valid = y[test_index]

        def augment(src, choice):
            if choice == 0:
                # Rotate 90
                src = np.rot90(src, 1)
            if choice == 1:
                # flip vertically
                src = np.flipud(src)
            if choice == 2:
                # Rotate 180
                src = np.rot90(src, 2)
            if choice == 3:
                # flip horizontally
                src = np.fliplr(src)
            if choice == 4:
                # Rotate 90 counter-clockwise
                src = np.rot90(src, 3)
            if choice == 5:
                # Rotate 180 and flip horizontally
                src = np.rot90(src, 2)
                src = np.fliplr(src)
            return src

        def train_generator():
            while True:
                for start in range(0, len(x_train), batch_size):
                    x_batch = []
                    end = min(start + batch_size, len(x_train))
                    y_batch = y_train[start:end]
                    for img in x_train[start:end]:
                        new_img = cv2.resize(img, img_size)
                        new_img = augment(new_img, np.random.randint(6))
                        x_batch.append(new_img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    y_batch = np.array(y_batch, np.uint8)
                    yield x_batch, y_batch

        def valid_generator():
            while True:
                for start in range(0, len(x_valid), batch_size):
                    x_batch = []
                    end = min(start + batch_size, len(x_valid))
                    y_batch = y_valid[start:end]
                    for img in x_valid[start:end]:
                        new_img = cv2.resize(img, img_size)
                        x_batch.append(new_img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    y_batch = np.array(y_batch, np.uint8)
                    yield x_batch, y_batch

        def test_generator():
            while True:
                for start in range(0, len(test), n_fold):
                    x_batch = []
                    end = min(start + n_fold, len(test))
                    for img in test[start:end]:
                        new_img = cv2.resize(img, img_size)
                        x_batch.append(new_img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    yield x_batch
                    
        callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, cooldown=1, 
                               verbose=1, min_lr=1e-7),
             ModelCheckpoint(filepath='inception.fold_' + str(i) + '.hdf5', verbose=1,
                             save_best_only=True, save_weights_only=True, mode='auto')]

        train_steps = len(x_train) / batch_size
        valid_steps = len(x_valid) / batch_size
        test_steps = len(test) / n_fold
        
        model = model

        model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy', metrics = ['accuracy'])

        model.fit_generator(train_generator(), train_steps, epochs=epochs, verbose=1, 
                            callbacks=callbacks, validation_data=valid_generator(), 
                            validation_steps=valid_steps)

        model.load_weights(filepath='inception.fold_' + str(i) + '.hdf5')

        
        print('----------------------------------------')
        print('Running train evaluation on fold {}'.format(i))
        train_score = model.evaluate_generator(train_generator(), steps=train_steps)        
        print('Running validation evaluation on fold {}'.format(i))
        valid_score = model.evaluate_generator(valid_generator(), steps=valid_steps)
        print('----------------------------------------')   
        
        print('Train loss: {:0.5f}\n Train acc: {:0.5f} for fold {}'.format(train_score[0],
                                                                            train_score[1], i))
        print('Valid loss: {:0.5f}\n Valid acc: {:0.5f} for fold {}'.format(valid_score[0],
                                                                            valid_score[1], i))
        print('----------------------------------------')

        train_scores.append(train_score[1])
        valid_scores.append(valid_score[1])
        print('Avg Train Acc: {:0.5f}\nAvg Valid Acc: {:0.5f} after {} folds'.format
              (np.mean(train_scores), np.mean(valid_scores), i))
        print('----------------------------------------')
        
        print('Running test predictions with fold {}'.format(i))        
        preds_test_fold = model.predict_generator(generator=test_generator(),
                                              steps=test_steps, verbose=1)[:, -1]

        preds_test += preds_test_fold

        print('\n\n')

        i += 1

        if i <= n_fold:
            print('Now beginning training for fold {}\n\n'.format(i))
        else:
            print('Finished training!')

    preds_test /= n_fold

    return preds_test

In [10]:
batch_size = 6
epochs = 50
n_fold = 3
img_size = (img_height, img_width)
kf = KFold(n_splits=n_fold, shuffle=True)

prediction = train_model(model, batch_size, epochs, img_size, X_train, 
                                target_train, X_test, n_fold, kf)

submit = pd.DataFrame({'id': id_test, 'is_iceberg': prediction.reshape((prediction.shape[0]))})
submit.to_csv('./submission.csv', index=False)

Epoch 1/50
179/178 [==============================] - 7s 41ms/step - loss: 0.6809 - acc: 0.5354 - val_loss: 2.4518 - val_acc: 0.5084
Epoch 2/50
179/178 [==============================] - 6s 32ms/step - loss: 0.6369 - acc: 0.5624 - val_loss: 0.5747 - val_acc: 0.7009
Epoch 3/50
179/178 [==============================] - 6s 32ms/step - loss: 0.5839 - acc: 0.6527 - val_loss: 0.5641 - val_acc: 0.6991
Epoch 4/50
179/178 [==============================] - 6s 31ms/step - loss: 0.5778 - acc: 0.6481 - val_loss: 0.5655 - val_acc: 0.7009
Epoch 5/50
179/178 [==============================] - 6s 32ms/step - loss: 0.5680 - acc: 0.6676 - val_loss: 0.5540 - val_acc: 0.7009
Epoch 6/50
179/178 [==============================] - 6s 31ms/step - loss: 0.5742 - acc: 0.6788 - val_loss: 0.5949 - val_acc: 0.5084
Epoch 7/50
179/178 [==============================] - 6s 32ms/step - loss: 0.5726 - acc: 0.6546 - val_loss: 0.5501 - val_acc: 0.7009
Epoch 8/50
179/178 [==============================] - 6s 31ms/step - 

Epoch 29/50
179/178 [==============================] - 6s 34ms/step - loss: 0.5394 - acc: 0.7011 - val_loss: 0.5363 - val_acc: 0.7215
Epoch 30/50
179/178 [==============================] - 6s 33ms/step - loss: 0.5409 - acc: 0.7030 - val_loss: 0.5360 - val_acc: 0.7196
Epoch 31/50
 57/178 [========>.....................] - ETA: 3s - loss: 0.5454 - acc: 0.7135

KeyboardInterrupt: 